In [1]:
import pandas as pd
import time
import re
import pathlib
import os
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [3]:
#https://chromedriver.chromium.org/downloads

In [4]:
driver_path = os.path.join(pathlib.Path().absolute(), 'chrome_driver\\chromedriver.exe')
driver = webdriver.Chrome(executable_path = driver_path, options=options)
driver.get('https://pv.servelelecciones.cl/')

In [5]:
%%time

df = pd.DataFrame([])

#PESTAÑA SUPERIOR: "Participación"
step1 = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[3]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step1))).click()

#PANEL LATERAL IZQUIERDO: "En Chile"
step2 = '/html/body/div[2]/div[1]/div[2]/div/ul/li[3]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step2))).click()

#PANEL LATERAL IZQUIERDO: "Distrito n°"
distrito = int(input('Indique el número del distrito que desea revisar (1-28): '))

def scraping(dfn, t1, t2, stepn):
    WebDriverWait(driver, t1).until(EC.element_to_be_clickable((By.XPATH, stepn))).click()
    time.sleep(t2)
    dfn = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
    dfn = dfn[dfn != '\n'].dropna(axis=0).reset_index(drop=True)
    return dfn

if distrito in range(1,29):
    step3 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[6]/option[{distrito + 1}]'
    df1 = pd.DataFrame([])
    df1 = scraping(df1, 5, 5, step3)
    df1[0] = df1[0].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
    
    for i in range(len(df1)):
        step4 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[9]/option[{i+2}]'
        df2 = pd.DataFrame([])
        df2 = scraping(df2, 5, 3, step4)
        df2[0] = df2[0].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))
        
        for j in range(len(df2)):
            step5 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[10]/option[{j+2}]'
            df3 = pd.DataFrame([])
            df3 = scraping(df3, 5, 3, step5)
            df3[0] = df3[0].apply(lambda x: "".join(re.split("( )", x)[0:len(re.split("( )", x))-8]))
            
            for k in range(len(df3)):
                step6 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[11]/option[{k+2}]'
                df4 = pd.DataFrame([])
                df4 = scraping(df4, 5, 3, step6)
                df4 = df4[0].str.split(expand=True)
                df4[5] = df3[0][k]
                df4[6] = df1[0][i]
                df4[7] = df2[0][j]
                df = pd.concat([df,df4])

    for m in range(1,4):
        df[m] = df[m].astype(int)

    df[4] = df[4].apply(lambda x: x.replace(',', '.')).astype('float64')
    columns = ['MESA', 'TOTALMESAS' , 'ELECTORES', 'EMITIDOS', 'PORCENTAJE', 'COLEGIO', 'COMUNA', 'CIRCUNSCRIPCION']
    df.columns = columns
    df = df.dropna(axis=0).reset_index(drop=True)
    path1 = os.path.join(pathlib.Path().absolute(), f'Distrito{distrito}')
    os.mkdir(path1)
    df.to_csv(f'Distrito{distrito}\Dist{distrito}_Pleb2020.csv', index=False)
    
else:
    sys.exit(f'No existe el distrito n°: {distrito}, elija uno entre 1-28')

Introducir el número del distrito que desea revisar (1-28)
3
Wall time: 5min 55s


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390 entries, 0 to 1389
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MESA             1390 non-null   object 
 1   TOTALMESAS       1390 non-null   int32  
 2   ELECTORES        1390 non-null   int32  
 3   EMITIDOS         1390 non-null   int32  
 4   PORCENTAJE       1390 non-null   float64
 5   COLEGIO          1390 non-null   object 
 6   COMUNA           1390 non-null   object 
 7   CIRCUNSCRIPCION  1390 non-null   object 
dtypes: float64(1), int32(3), object(4)
memory usage: 70.7+ KB


In [7]:
df

,MESA,TOTALMESAS,ELECTORES,EMITIDOS,PORCENTAJE,COLEGIO,COMUNA,CIRCUNSCRIPCION
0,113M,1,347,213,61.38,COLEGIO ANTOFAGASTA,ANTOFAGASTA,ANTOFAGASTA NORTE
1,114M,1,345,216,62.61,COLEGIO ANTOFAGASTA,ANTOFAGASTA,ANTOFAGASTA NORTE
2,115M,1,347,225,64.84,COLEGIO ANTOFAGASTA,ANTOFAGASTA,ANTOFAGASTA NORTE
3,116M,1,341,203,59.53,COLEGIO ANTOFAGASTA,ANTOFAGASTA,ANTOFAGASTA NORTE
4,282,1,349,153,43.84,COLEGIO ANTOFAGASTA,ANTOFAGASTA,ANTOFAGASTA NORTE
...,...,...,...,...,...,...,...,...
1385,29M,1,332,173,52.11,LICEO POLITECNICO DIEGO PORTALES P.,TOCOPILLA,TOCOPILLA
1386,30M,1,331,171,51.66,LICEO POLITECNICO DIEGO PORTALES P.,TOCOPILLA,TOCOPILLA
1387,31M,1,328,185,56.40,LICEO POLITECNICO DIEGO PORTALES P.,TOCOPILLA,TOCOPILLA
1388,32M,1,333,192,57.66,LICEO POLITECNICO DIEGO PORTALES P.,TOCOPILLA,TOCOPILLA
